In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
import urllib
from bs4 import BeautifulSoup
import requests

In [81]:
url = "https://www.datacamp.com/community/tutorials"
page = requests.get(url)
#html = urlopen(url)

page=urllib.request.Request(url,headers={'User-Agent': 'Mozilla/5.0'}) 
infile=urllib.request.urlopen(page).read()
data = infile.decode('ISO-8859-1') # Read the content as string decoded with ISO-8859-1

#print(data)


soup = BeautifulSoup(data, 'html.parser')

pages = [i.text for i in soup.find_all('a') if 'community/tutorials?page=' in str(i)]
lastpage = pages[-1]

In [82]:
print(lastpage)

29


In [83]:
description=[]
upvote=[]
author=[]
publishdate=[]
title=[]

In [84]:
for cp in np.arange(1,int(lastpage)+1):
    url = "https://www.datacamp.com/community/tutorials?page=" + str(cp)
    #html = urlopen(url)
    
    page=urllib.request.Request(url,headers={'User-Agent': 'Mozilla/5.0'}) 
    infile=urllib.request.urlopen(page).read()
    data = infile.decode('ISO-8859-1') # Read the content as string decoded with ISO-8859-1
    soup = BeautifulSoup(data, 'html')
    description.append([i.text for i in soup.find_all(class_='jsx-2625178925 blocText description')])
    upvote.append([i.text for i in soup.find_all(class_='jsx-1727309017 voted')])
    author.append([i.text for i in soup.find_all(class_='jsx-886169423 name')])
    publishdate.append([i.text for i in soup.find_all(class_='jsx-886169423 date')])
    title.append([i.text for i in soup.find_all(class_='jsx-2625178925 blue')])
print ("Done!")

Done!


In [85]:
descriptionflat = [y for x in description for y in x]
upvoteflat = [y for x in upvote for y in x]
authorflat = [y for x in author for y in x]
publishdateflat = [y for x in publishdate for y in x]
titleflat = [y for x in title for y in x]
publishdateformatted = [datetime.strptime(re.sub('rd, ', ', ', re.sub('st, ', ', ', re.sub('nd, ', ', ', re.sub('th, ',', ',a)))), "%B %d, %Y") for a in publishdateflat]

In [86]:
print(len(descriptionflat), len(upvoteflat), len(authorflat),len(publishdateflat),len(titleflat))

429 429 429 429 429


In [90]:
cdata = {"author":authorflat,"publishdate":publishdateformatted,"title":titleflat,"description":descriptionflat,"upvote":upvoteflat}
df = pd.DataFrame(data = cdata)
df.to_csv("c:\\users\\Turabi Traders\\documents\\datacamp130818.csv", header=True, index=False)

In [91]:
datacamp = pd.read_csv("c:\\users\\Turabi Traders\\documents\\datacamp130818.csv", parse_dates=["publishdate"], infer_datetime_format=True)

In [92]:
datacamp.shape

(429, 5)

In [93]:
datacamp.head()

,author,publishdate,title,description,upvote
0,DataCamp Team,2020-10-23,Installing Jupyter Notebook,,44
1,DataCamp Team,2020-10-23,Joins in SQL,,19
2,DataCamp Team,2020-10-23,What is SQL?,,20
3,DataCamp Team,2020-10-23,PCH in R,,2
4,DataCamp Team,2020-10-23,Python String format(),,12


In [95]:
datacamp.tail()

,author,publishdate,title,description,upvote,publishyymm,posts
424,Karlijn Willems,2014-06-09,Decimal Comma or Point? A googleVis visualization,Decimal comma or decimal point? A handy tool t...,3,2014-Jun,1
425,Karlijn Willems,2014-06-03,What Is the Best Statistical Programming Langu...,The infograph 'Statistical Language Wars' comp...,6,2014-Jun,1
426,Karlijn Willems,2014-03-31,Data Science Cartoons: a Top Seven List by Dat...,The 7 funniest data science cartoons on the we...,10,2014-Mar,1
427,Karlijn Willems,2014-03-17,R posts on Stack Overflow | A Top 5 selection ...,Stack Overflow is a useful resource for techni...,4,2014-Mar,1
428,Karlijn Willems,2013-12-04,Coursera Courses on R: Complete List Ranked by...,A list of all Coursera courses on R. These cou...,7,2013-Dec,1


In [94]:
datacamp['publishyymm'] = datacamp['publishdate'].dt.strftime("%Y-%b")
datacamp["posts"] = 1